In [ ]:
##########################################################
# This is my final project from the 2021 Python Basics course
# at the Warsaw School of Economics.

##############
# The task was to write a program to perform the following functions:
# 1. reads the parameters and prices of cell phones listed for sale on euroagd.com.pl and MediaMarkt.pl
# 2. writes down the prices of cell phones
# 3. identifies the cell phones appearing on both sites.
##########################################################



from urllib.request import Request, urlopen
import  re
import pandas as pd
from unicodedata import normalize
from bs4 import BeautifulSoup as bs




### webpage www.euro.com.pl ###

# starting adress
adresEuro = 'https://www.euro.com.pl/telefony-komorkowe,d1.bhtml'
adresKolejnejStronyEuro = 'https://www.euro.com.pl/telefony-komorkowe,d1,strona-2.bhtml'
#regex
pamięćRegex = re.compile(r'\d{1,2}/\d{2,3}GB|\d{1,2}\+\d{2,3}GB|\d{2,3}GB|\d{1,2}/\d{2,3}|\d{1,2}\+\d{2,3}')
kolorRegex = re.compile(r'\(\w*\s?\w*\)|\(\w*-\w*\)')
pamięć2Regex = re.compile(r'\d{1,2}')


def telefony_i_parametry_euro(css_class):
    return css_class == 'product-name' or css_class == 'attribute-value' or css_class == 'price-normal selenium-price-normal'


# downloading the number of pages with offers
req = Request(adresEuro, headers={'User-Agent': 'XYZ/3.0'})
webpage = urlopen(req, timeout=10).read()
soup = bs(webpage, 'html.parser')

numeryStronEuro = soup.find_all('a', class_='paging-number')
for i in numeryStronEuro:
    a = i.get_text()
    aList = a.split()
    
aList = [int(i) for i in aList]
liczbaStronEuro = int(max(aList))



modeleEuro = []
parametry = []
telefonyEuro = {}
counter = 2


###########   webscraping from EURO webpage
# main loop - x pages on Euro webpage
for x in range(liczbaStronEuro):
    if x != 0:
        if x != 1:
            adresKolejnejStronyEuro = adresKolejnejStronyEuro.replace(str(counter), str(counter+1))
            counter += 1
        
        req = Request(adresKolejnejStronyEuro, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=20).read()
        soup = bs(webpage, 'html.parser')
        
#         print('modeleEuro', len(modeleEuro))
#         print('telefonyEuro', len(telefonyEuro))
#         print(adresKolejnejStronyEuro)

    
    nazwyEuro = soup.find_all('a', class_='js-save-keyword')
    telefonySoup = soup.find_all(class_=telefony_i_parametry_euro)
    


    # creating list of models
    for i in nazwyEuro:
        a = i.get_text(strip = True)
        aList = a.split()
        if aList:
            a = ' '.join(aList)
            if not 'opini' in a:
                   if not a in modeleEuro:
                    a = normalize('NFKD', a)
                    modeleEuro.append(a)
    
    
    
    
    # creating dictionary of phones and it's parameters
    for i in telefonySoup:
        a = i.get_text(strip=True)
        aList = a.split()
        if aList:
            a = ' '.join(aList)
            if not 'opini' in a:
                a = normalize('NFKD', a)
                # element is phone name
                if a in modeleEuro:
                    klucz = a
                    # deleting memory parameters from the phone name
                    pamięćTelefonu = pamięćRegex.search(klucz)
                    if pamięćTelefonu:
                        pamięćTelefonu = pamięćTelefonu.group()
                        klucz = klucz.replace(' ' + pamięćTelefonu, '')
                        # unification of memory storage (format \d\d/\d\d\d)
                        for x in range(len(pamięćTelefonu)):
                            if pamięćTelefonu[x] == '/':
                                pass
                            elif pamięćTelefonu[x] == '+':
                                pamięćTelefonu = pamięćTelefonu.replace('+', '/')            
                        pamięćTelefonu = pamięćTelefonu.replace('GB', '')
                        klucz = klucz + ' ' + pamięćTelefonu
                    # unifying the color name in the phone name
                    kolorTelefonu = kolorRegex.search(klucz)
                    if kolorTelefonu:
                        kolorTelefonu = kolorTelefonu.group()
                        klucz = klucz.replace(' ' + kolorTelefonu, '')
                        kolorTelefonu = kolorTelefonu.replace('(', '')
                        kolorTelefonu = kolorTelefonu.replace(')', '')
                        parametry.append(kolorTelefonu)
                        klucz = klucz + ' ' + kolorTelefonu

                    # adding a key with parameters to the dictionary
                    parametry.insert(0, parametryCena)
                    telefonyEuro[klucz] = parametry
                    parametry = []


                # element is price
                elif a.endswith('zł'):
                    parametryCena = normalize('NFKD', a)
                    parametryCena = parametryCena.replace(' ', '')
                    parametryCena = parametryCena.replace('zł', ' zł')



                #element is parameter    
                elif a:
                    if a not in parametry:
                        if 'GB' in a:
                            p = a.split()
                            for i in range(len(p)):
                                if pamięć2Regex.search(p[i]):
                                    if p[i+1] == 'GB' or p[i+1] == 'MB':
                                        par = p[i] + ' ' + p[i+1]
                                        parametry.append(par)
                        else:
                            parametry.append(a)          


### webpage mediamarkt.pl ###

# starting adress
adresMedia = 'https://mediamarkt.pl/telefony-i-smartfony/smartfony?sort=name_asc&limit=20&page=1'

req = Request(adresMedia, headers={'User-Agent': 'XYZ/3.0'})
webpage = urlopen(req, timeout=10).read()
soup = bs(webpage, 'html.parser')

# downloading the number of pages with offers
numeryStronMedia = soup.find_all('span', class_='m-pagination_count')
for i in numeryStronMedia:
    a = i.get_text(strip=True)
    aList = a.split()

liczbaStronMedia = int(aList[1])



#regex
pamięćRegex = re.compile(r'''\d{1,2}\s*/\s*\d{2,3}\s*GB|\d{1,2}\s*/\s*\d{2,3}|\d{1,2}\s*GB\s*/\s*\d{2,3}\s*GB|\d{1,2}\s*\+\s*\d{2,3}\s*GB|\d{1,2}\s*\+\s*\d{2,3}|\d{1,2}\s*GB\s*\+\s*\d{2,3}\s*GB|\d{1,3}\s*GB''')
numerModeluRegex = re.compile(r'[A-Z0-9]*[-/][A-Z0-9]*')

def telefony_i_parametry(css_class):
    return css_class == 'b-ofr_headDataTitle' or css_class == 'b-ofr_boxDataItem is-tech' or css_class == 'm-priceBox_price'

modeleMedia = []
parametry = []
adresKolejnejStronyMedia = adresMedia
telefonyMedia = {}
counter = 1

###########   webscraping from Media webpage
# main loop - x pages on Media webpage
for x in range(liczbaStronMedia):
    if x != 0:
        adresKolejnejStronyMedia = adresKolejnejStronyMedia.replace(str(counter), str(counter+1))
        counter += 1
        
        req = Request(adresKolejnejStronyMedia, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=20).read()
        soup = bs(webpage, 'html.parser')

    
    nazwyMedia = soup.find_all('a', class_='b-ofr_headDataTitle')
    telefonySoupMedia = soup.find_all(class_=telefony_i_parametry)

     
#     print(adresKolejnejStronyMedia)    
    # creating list of phone models
    for i in nazwyMedia:
        
        a = i.get_text()
        aList = a.split()
        if aList:
            a = ' '.join(aList)
            if not a in modeleMedia:
                a = normalize('NFKD', a)
                modeleMedia.append(a)
    
    
    
    
    # creating dictionary of phones and it's parameters
    for i in telefonySoupMedia:
        a = i.get_text()
        #element is phone name
        if a in modeleMedia:
            klucz = normalize('NFKD', a)
            # deleting 'Smartfon' from name
            klucz = klucz.replace('Smartfon ', '')
            # deletig colour and model number
            for i in reversed(range(0, len(klucz))):
                if klucz[i] == 'B' and klucz[i-1] == 'G':
                    numb = i + 1
                    break
                else:
                    numb = 0

            # memory in the name   
            if numb != 0:
                kolorTelefonu = klucz[numb+1:]
                klucz = klucz[0:numb]
                kolorLista = kolorTelefonu.split()
                kolorTelefonu = kolorLista[0]

            # no memory in the name - deleting model number
            elif numb == 0:
                try:
                    numerModelu = numerModeluRegex.search(klucz).group()
                    klucz = klucz.replace(' ' + numerModelu, '')
                except:
                    pass
                kolorTelefonu = klucz.split()[-1]
                klucz = klucz.replace(' ' + kolorTelefonu, '')
                
#             parametry.append(kolorTelefonu)    
            klucz += ' ' + kolorTelefonu
    
            # deleting memory parameters
            pamięćTelefonu = pamięćRegex.search(klucz)
            if pamięćTelefonu:
                pamięćTelefonu = pamięćTelefonu.group()
                klucz = klucz.replace(' ' + pamięćTelefonu, '')
                # unifying memory format (format \d\d/\d\d\d)
                for x in range(len(pamięćTelefonu)):
                    if pamięćTelefonu[x] == '/':
                        pass
                    elif pamięćTelefonu[x] == '+':
                        pamięćTelefonu = pamięćTelefonu.replace('+', '/')            
                pamięćTelefonu = pamięćTelefonu.replace('GB', '')
                klucz = klucz + ' ' + pamięćTelefonu
                
            klucz = klucz.replace('(', ' ')
            klucz = klucz.replace(')', ' ')
            klucz = klucz.replace('  ', ' ')
                
                


        #element is a parameter    
        else:
            #element is a parameter   
            if i.strong:
                for col in i.find_all('strong'):
                    parametr = col.get_text(strip=True)
                    parametr = normalize('NFKD', parametr)
                    parametr = re.sub("\s+", " ", parametr)
                    parametry.append(parametr)
                    
            #element is a price
            else:
                parametryCena = i.get_text(strip=True)
                parametryCena = normalize('NFKD', parametryCena)
                parametryCena = parametryCena.replace(',-zł', ' zł')
                
                
                 # addin key with parameters to dictionary   
                parametry.insert(0, parametryCena)
                parametry.append(kolorTelefonu)
                telefonyMedia[klucz] = parametry
                parametry = []
              

    
    
                   



przekątnaEuroRe = re.compile(r'^[1-6],\d{1,2}')
przekątnaMediaRe = re.compile(r'^[1-6]\.\d{1,2}')
rozdzielczośćRe = re.compile(r'\d{3,4}\sx\s\d{3,4}')




# comparing phones from mediamarkt and euro.com
wspólne = []
for keyEuro in telefonyEuro:
    najlepszeDopasowanie = ['', 0, 0]
    slowaEuro = keyEuro.split()
    for s in range(len(slowaEuro)):
        slowaEuro[s] = slowaEuro[s].lower()
    for keyMedia in telefonyMedia:
        dopasowanieNazwa = 0
        dopasowaniePamięć = 0
        counterPamięć = 0
        dopasowaniePrzekątna = 0
        dopasowanieRozdzielczość = 0
        dopasowanieKolor = 0

        slowaMedia = keyMedia.split()
        #to lowercase
        for s in range(len(slowaMedia)):
            slowaMedia[s] = slowaMedia[s].lower()
        if slowaEuro[0].lower() == slowaMedia[0].lower():
            for slowo in slowaEuro:
                if slowo in slowaMedia:
                    dopasowanieNazwa += 1
            if dopasowanieNazwa > najlepszeDopasowanie[1]:
                najlepszeDopasowanie[0] = keyMedia
                najlepszeDopasowanie[1] = dopasowanieNazwa
                
    if najlepszeDopasowanie[0]:

        if dopasowanieNazwa == len(slowaEuro):
            najlepszeDopasowanie[2] = True
            
        else:
            for parE in telefonyEuro[keyEuro]:
                if 'GB' in parE or 'MB' in parE:
                        counterPamięć += 1
                for parM in telefonyMedia[najlepszeDopasowanie[0]]:
                    
                    # Ram i Rom
                    if 'GB' in parE or 'MB' in parE:
                        if parE == parM:
                            dopasowaniePamięć += 1
                            
                    
                    # diagonal
                    if przekątnaEuroRe.search(parE):  
                        if przekątnaMediaRe.search(parM):
                            przekątnaEuro = przekątnaEuroRe.search(parE).group()
                            przekątnaMedia = parM.replace('.', ',')
                            if przekątnaEuro == przekątnaMedia:
                                dopasowaniePrzekątna = 1

                                
                    # resolution
                    if rozdzielczośćRe.search(parE):
                        if rozdzielczośćRe.search(parM):
                            rozdzielczośćEuro = rozdzielczośćRe.search(parE).group()
                            rozdzielczośćEuro = sorted(rozdzielczośćEuro.split())
                            rozdzielczośćMedia = rozdzielczośćRe.search(parM).group()
                            rozdzielczośćMedia = sorted(rozdzielczośćMedia.split())
                            if rozdzielczośćEuro == rozdzielczośćMedia:
                                dopasowanieRozdzielczość = 1

                                
            # color
            if telefonyEuro[keyEuro][-1].lower() == telefonyMedia[najlepszeDopasowanie[0]][-1].lower():
                dopasowanieKolor = 1


            elif telefonyEuro[keyEuro][-1].upper() == 'PRODUCT' or telefonyEuro[keyEuro][-1].lower() == 'product red' or telefonyEuro[keyEuro][-1].lower() == 'czerwony' and telefonyMedia[najlepszeDopasowanie[0]][-1].upper() == '(PRODUCT)RED':
                dopasowanieKolor = 1




            if dopasowaniePamięć == counterPamięć and dopasowanieKolor and dopasowanieRozdzielczość:
                najlepszeDopasowanie[2] = True


    if najlepszeDopasowanie:
        if najlepszeDopasowanie[2]:
            wspólne.append([keyEuro, najlepszeDopasowanie[0]])

            
            

                        



frame = pd.DataFrame()

frame['RTV Euro AGD'] = []
frame['cena Euro'] = []
frame['Media Markt'] = []
frame['cena Media'] = []
frame['różnica w cenie'] = []



columns = list(frame)
data =[]

for para in wspólne:
    cenaEuro = float(telefonyEuro[para[0]][0].replace(' zł', ''))
    cenaMedia = float(telefonyMedia[para[1]][0].replace(' zł', ''))
    różnica = max(cenaEuro, cenaMedia) - min(cenaEuro, cenaMedia)
    values = [para[0], telefonyEuro[para[0]][0], para[1], telefonyMedia[para[1]][0], różnica]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
#     print(a_dictionary)
    data.append(a_dictionary)
    
    
frame = frame.append(data, True)


pd.set_option("display.max_rows", None, "display.max_columns", None)
frame
